# Minimum Fisher Regularization (MFR) tomography


## Diffinition

MFR was firstly introduced to solve the ill-posedness of the tomography problem for tokamak plasma.
The MFR tomography is formulated as a constrained optimization problem:

$$
\begin{align}
\min_{\mathbf{x}} \quad &  \left\| \mathbf{A} \mathbf{x} - \mathbf{b} \right\|_2^2
    + \lambda\cdot \mathbf{x}^\mathsf{T} \mathbf{H} \mathbf{x}\\
\mathbf{H} & = \sum_{i, j} \alpha_{i,j} \mathbf{D}_i^\mathsf{T} \mathbf{W} \mathbf{D}_j\\
\mathbf{W} & = \rm{diag}
    \left(
        \cdots,\frac{1}{\max\left\{\mathbf{x}_\mathit{i}, \epsilon_0\right\}},\cdots
    \right),
\end{align}
$$

where $\mathbf{x}$ is the vector of the unknowns, $\mathbf{A}$ is the geometry matrix, $\mathbf{b}$ is the measured data, $\lambda$ is the regularization parameter, $\mathbf{H}$ is the regularization matrix, $\mathbf{D}_{i,j}$ is derivative matrices along the $i$ or $j$ coordinate direction, $\alpha_{i, j}$ is the anisotropic parameter, $\mathbf{W}$ is the weight matrix, $\mathbf{x}_\mathit{i}$ is the $i$-th element of $\mathbf{x}$, and $\epsilon_0$ is a small positive number to avoid division by zero and to push the solution to be positive.

The MFR method is the iterative method, and the iteration formula is:

1. Put $\mathbf{x}^{(0)} = \mathbf{1}$ as the initial guess;
2. Compute $\mathbf{H}^{(k)}$ with $\mathbf{x}^{(k)}$;
3. Solve $\mathbf{x}^{(k+1)}$ optimizing regularization parameter $\lambda$ by non-iterative inversion method;

the iteration between step 2 and 3 is repeated until the convergence is reached or the maximum iteration number is reached.

Several non-iterative inversion methods (e.g. L-curve method) can be used in step 3.


## Example Tomography Problem

As the example tomography problem, we consider the bolometer measurement and use the geometry matrix
calculated at `cherab.core` [document](https://www.cherab.info/demonstrations/bolometry/geometry_matrix_with_raytransfer.html#bolometer-geometry-raytransfer).


In [ ]:
import pickle
from pathlib import Path
from pprint import pprint

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize, SymLogNorm
from matplotlib.ticker import LogFormatterSciNotation, SymmetricalLogLocator
from mpl_toolkits.axes_grid1 import ImageGrid

from cherab.inversion import Mfr
from cherab.inversion.derivative import compute_dmat
from cherab.inversion.tests import __path__ as test_path

plt.rcParams["figure.dpi"] = 150
TEST_DATA_PATH = Path(test_path[0]) / "data"

In [ ]:
# Load the results of the cherab.core demo
grid_data = np.load(TEST_DATA_PATH / "raytransfer_grid_data.npz")

# Extract the data
grid_centres = grid_data["grid_centres"]
voxel_map = grid_data["voxel_map"].squeeze()
mask = grid_data["mask"].squeeze()
gmat = grid_data["sensitivity_matrix"]

# Extract grid limits
rmin, rmax = grid_centres[0, 0, 0], grid_centres[-1, 0, 0]
zmin, zmax = grid_centres[0, 0, 1], grid_centres[0, -1, 1]

print(f"{grid_centres.shape = }")
print(f"{voxel_map.shape = }")
print(f"{mask.shape = }")
print(f"{gmat.shape = }")
print(f"gmat density: {np.count_nonzero(gmat) / gmat.size:.2%}")

The geometry matrix density is 8.35% so this problem is known as the sparse problem as well.


### Show the geometry matrix


In [ ]:
sensitivity_2d = np.full(voxel_map.shape, np.nan)
sensitivity_2d[mask[:, :]] = gmat.sum(axis=0)

fig, ax = plt.subplots()
image = ax.imshow(
    sensitivity_2d.T,
    origin="lower",
    cmap="Reds",
    extent=(rmin, rmax, zmin, zmax),
)
cbar = plt.colorbar(image)
cbar.set_label("m$^3\\cdot$str")
ax.set_title("Geomtery matrix")
ax.set_xlabel("r")
ax.set_ylabel("z");

### Define phantom emission profile

As the test emission profile, we define the following phantom emission profile which is used in the `cherab.core`
[document](https://www.cherab.info/demonstrations/bolometry/inversion_with_raytransfer.html#bolometer-raytransfer-inversion) as well.


In [ ]:
PLASMA_AXIS = np.array([1.5, 1.5])
LCFS_RADIUS = 1
RING_RADIUS = 0.5

RADIATION_PEAK = 1
CENTRE_PEAK_WIDTH = 0.05
RING_WIDTH = 0.025


def emission_function_2d(rz_point: np.ndarray) -> float:
    direction = rz_point - PLASMA_AXIS
    bearing = np.arctan2(direction[1], direction[0])

    # calculate radius of coordinate from magnetic axis
    radius_from_axis = np.hypot(*direction)
    closest_ring_point = PLASMA_AXIS + (0.5 * direction / radius_from_axis)
    radius_from_ring = np.hypot(*(rz_point - closest_ring_point))

    # evaluate pedestal-> core function
    if radius_from_axis <= LCFS_RADIUS:
        central_radiatior = RADIATION_PEAK * np.exp(-(radius_from_axis**2) / CENTRE_PEAK_WIDTH)

        ring_radiator = (
            RADIATION_PEAK * np.cos(bearing) * np.exp(-(radius_from_ring**2) / RING_WIDTH)
        )
        ring_radiator = max(0, ring_radiator)

        return central_radiatior + ring_radiator
    else:
        return 0


# Create a phantom vector
phantom = np.zeros(gmat.shape[1])
for i in range(phantom.shape[0]):
    (row,), (col,) = np.where(voxel_map == i)
    phantom[i] = emission_function_2d(grid_centres[row, col])

# Create a 2D phantom
phantom_2d = np.full(voxel_map.shape, np.nan)
phantom_2d[mask[:, :]] = phantom

In [ ]:
fig, ax = plt.subplots()
image = ax.imshow(
    phantom_2d.T,
    origin="lower",
    cmap="Reds",
    extent=(rmin, rmax, zmin, zmax),
)
cbar = plt.colorbar(image)
cbar.set_label("W/m$^3$")
ax.set_title("Phantom emissivity")
ax.set_xlabel("r")
ax.set_ylabel("z");

### Compute the measurement data

The bolometer measurement data should be calculated by the ray-tracing method, however, we generate
the measurement data by the multiplication of the geometry matrix and the phantom profile.
As the noise, we add the Gaussian noise with the standard deviation of 1% of the maximum value of the
measurement data.


In [ ]:
gmat /= 4.0 * np.pi  # devided by 4pi str to use with power measurements in [W]

In [ ]:
data = gmat @ phantom

rng = np.random.default_rng()
data_w_noise = data + rng.normal(0, data.max() * 1.0e-2, data.size)

In [ ]:
plt.plot(data, label="w/o noise")
plt.plot(data_w_noise, "--", label="w/ noise")
plt.legend()
plt.xlabel("channel of bolometers")
plt.ylabel("Power [W]");

## MFR Tomography


### Prepare for the MFR tomography

Before do the MFR tomography, we need to create derivative matrices. Here $\mathbf{H}$ is defined as follows:

$$
\mathbf{H} \equiv \mathbf{D}_r^\mathsf{T} \mathbf{W} \mathbf{D}_r
    + \mathbf{D}_z^\mathsf{T} \mathbf{W} \mathbf{D}_z
$$

where $\mathbf{D}_r$ and $\mathbf{D}_z$ are derivative matrices along the $r$ and $z$ coordinate direction, respectively.


In [ ]:
dmat_r = compute_dmat(voxel_map, kernel_type="r")
dmat_z = compute_dmat(voxel_map, kernel_type="z")

dmat_pair = [(dmat_r, dmat_r), (dmat_z, dmat_z)]
pprint(dmat_pair)

Some parameters are defined as follows:


In [ ]:
num_mfi = 4  # number of MFR iterations
eps = 1e-6  # minimum of solution
bounds = (-40, -10)  # bounds of log10 of regularization parameter
tol = 1e-2  # tolerance of MFR iteration

### MFR tomographic reconstruction

Let us do the MFR tomography. `cherab.inversion` offers the `Mfr` class and simple `solve` method.
The `store_regularizers` parameter of the `solve` method is used to store the regularizer object
like `Lcurve` instance at each MFR iteration.


In [ ]:
mfr = Mfr(gmat, dmat_pair, data_w_noise)
sol, stats = mfr.solve(
    miter=num_mfi, eps=eps, bounds=bounds, tol=tol, store_regularizers=True, use_gpu=False
)

## Evaluate the tomographic reconstruction


### Compare the tomographic reconstruction with the phantom profile


In [ ]:
vmin = min(sol.min(), phantom.min())
vmax = max(sol.max(), phantom.max())

fig = plt.figure(figsize=(10, 5))
grids = ImageGrid(fig, 111, nrows_ncols=(1, 2), axes_pad=0.05, cbar_mode="single")
norm = Normalize(vmin=vmin, vmax=vmax)
for ax, value, label in zip(grids, [phantom, sol], ["Phantom", "Reconstruction"], strict=True):
    image = np.full(voxel_map.shape, np.nan)
    image[mask[:, :]] = value

    ax.imshow(
        image.T,
        origin="lower",
        cmap="Reds",
        extent=(rmin, rmax, zmin, zmax),
        norm=norm,
    )
    ax.set_title(label)
    ax.set_xlabel("r")

grids[0].set_ylabel("z")
mappable = ScalarMappable(cmap="Reds", norm=norm)
cbar = plt.colorbar(mappable, cax=grids.cbar_axes[0])
cbar.set_label("Emissivity [W/m$^3$]");

In [ ]:
print(
    f"The relative error of both emissivities is {np.linalg.norm(sol - phantom) / np.linalg.norm(phantom):.2%}"
)

### The iteration history

Let us see the each iteration solution. To clarify the negative values, we plot solutions with the
logarithmic scale.


In [ ]:
# Stored regularizer files
reg_files = list(Path().cwd().glob("regularizer_*.pickle"))
reg_files = sorted(reg_files, key=lambda x: int(x.stem.split("_")[-1]))

# Load each solutions
sols = []
for file in reg_files:
    with open(file, "rb") as f:
        reg = pickle.load(f)

    sols.append(reg.inverted_solution(reg.lambda_opt))


# set vmin and vmax for all solutions
vmin = min(sol.min() for sol in sols)
vmax = max(sol.max() for sol in sols)


# Plot the solutions
fig = plt.figure(figsize=(10, 5))
grids = ImageGrid(
    fig,
    111,
    nrows_ncols=(1, len(sols)),
    axes_pad=0.05,
    cbar_mode="single",
    cbar_location="right",
)

absolute = max(abs(vmax), abs(vmin))
linear_width = 1e-1  # linear region width
norm = SymLogNorm(linthresh=linear_width, vmin=-1 * absolute, vmax=absolute)
i = 0
for ax, sol in zip(grids, sols, strict=True):
    sol_2d = np.full(voxel_map.shape, np.nan)
    sol_2d[mask[:, :]] = sol

    ax.imshow(
        sol_2d.T,
        origin="lower",
        cmap="bwr",
        extent=(rmin, rmax, zmin, zmax),
        norm=norm,
    )
    ax.set_title(f"{i}. MFR iteration")
    ax.set_xlabel("r")
    i += 1

grids[0].set_ylabel("z")
mappable = ScalarMappable(norm=norm, cmap="bwr")
cbar = plt.colorbar(mappable, cax=grids.cbar_axes[0])
cbar.set_label("Emissivity [W/m$^3$]")
fmt = LogFormatterSciNotation(linthresh=linear_width)
major_locator = SymmetricalLogLocator(linthresh=linear_width, base=10)
minor_locator = SymmetricalLogLocator(
    linthresh=linear_width, base=10, subs=tuple(np.arange(0.1, 1.0, 0.1))
)
cbar.ax.yaxis.set_offset_position("left")
cbar.ax.yaxis.set_major_formatter(fmt)
cbar.ax.yaxis.set_major_locator(major_locator)
cbar.ax.yaxis.set_minor_locator(minor_locator);

### L-curve and its curvature plot


In [ ]:
lcurve = stats["regularizer"]
bounds = (-30, -20)
fig, axes = plt.subplots(1, 2, dpi=150, figsize=(9, 4), constrained_layout=True)
lcurve.plot_L_curve(fig, axes[0], bounds=bounds)
lcurve.plot_curvature(fig, axes[1], bounds=bounds);

### Compare the measurement powers

In [ ]:
back_calculated_measurements = gmat @ sols[-1]

plt.plot(data, label="Matrix-based measurements")
plt.plot(back_calculated_measurements, linestyle="--", label="Back-calculated from inversion")
plt.legend()
plt.xlabel("channel of bolometers")
plt.ylabel("Power [W]");

The measured power power calculated by multiplying the geomtery matrix by the emission vector and the back-calculated power calculated by multiplying the geometry matrix by the inverted emissivity are all in good agreement


In [ ]:
print(
    f"The relative error of power measurements is {np.linalg.norm(data - back_calculated_measurements) / np.linalg.norm(data):.2%}"
)